In [3]:
import os

In [4]:
%pwd

'c:\\Project\\Insurance-ml-Project\\research'

In [5]:
os.chdir("../")


In [6]:
%pwd

'c:\\Project\\Insurance-ml-Project'

In [7]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class DataTransformationConfig:
    root_dir: Path
    data_path: Path

In [8]:

from src.mlProject.constants import *
from src.mlProject.utils.common import read_yaml, create_directories

In [9]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])

    
    def get_data_transformation_config(self) -> DataTransformationConfig:
        config = self.config.data_transformation

        create_directories([config.root_dir])

        data_transformation_config = DataTransformationConfig(
            root_dir=config.root_dir,
            data_path=config.data_path,
        )

        return data_transformation_config

In [10]:
import os
from src.mlProject.logging import logger
from sklearn.model_selection import train_test_split
import pandas as pd
from sklearn.preprocessing import OneHotEncoder
import joblib
import os

In [15]:
class DataTransformation:
    def __init__(self, config):
        self.config = config
        self.encoder = None

    def train_test_split_and_encode(self):
        data = pd.read_csv(self.config.data_path)

        # Define categorical columns
        categorical_cols = ['sex', 'smoker', 'region']
        target_column = 'expenses'

        # Encode categorical columns
        data_encoded = pd.get_dummies(data, columns=categorical_cols, drop_first=True)

        # Ensure 'expenses' column is included in the encoded data
        if target_column not in data_encoded.columns:
            raise ValueError(f"'{target_column}' column is missing in the encoded data")

        # Split the data into features (X) and target variable (y)
        X = data_encoded.drop(columns=[target_column])
        y = data_encoded[target_column]

        # Check if the lengths of X and y match
        if len(X) != len(y):
            raise ValueError("Length of features (X) does not match length of target variable (y)")

        # Split the data into training and test sets
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

        # Save the encoder
        encoder_path = os.path.join(self.config.root_dir, "encoder.pkl")
        joblib.dump(self.encoder, encoder_path)

        # Save the transformed data as CSV files
        train_csv_path = os.path.join(self.config.root_dir, "train.csv")
        test_csv_path = os.path.join(self.config.root_dir, "test.csv")
        pd.concat([X_train, y_train], axis=1).to_csv(train_csv_path, index=False)
        pd.concat([X_test, y_test], axis=1).to_csv(test_csv_path, index=False)

        return train_csv_path, test_csv_path, encoder_path


In [16]:
try:
    config = ConfigurationManager()
    data_transformation_config = config.get_data_transformation_config()
    data_transformation = DataTransformation(config=data_transformation_config)
    data_transformation.train_test_split_and_encode()
except Exception as e:
    raise e

[2024-02-22 02:37:30,493: INFO: common: yaml file: config\config.yaml loaded successfully]
[2024-02-22 02:37:30,516: INFO: common: yaml file: params.yaml loaded successfully]
[2024-02-22 02:37:30,527: INFO: common: yaml file: schema.yaml loaded successfully]
[2024-02-22 02:37:30,534: INFO: common: created directory at: artifacts]
[2024-02-22 02:37:30,543: INFO: common: created directory at: artifacts/data_transformation]
